In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, ZeroPadding2D, Input, Activation
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import activations
import tensorflow as tf
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Загрузка токена аутентификации
file = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
# Просматриваем содержимое текущей директории
!ls

kaggle.json  mnist_test.csv  mnist_train.csv  sample_data


In [4]:
# Копирование токена в каталог .kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
# Загружаем данные с сайта Kaggle
!kaggle competitions download -c digit-recognizer

100% 15.3M/15.3M [00:01<00:00, 21.7MB/s]
100% 15.3M/15.3M [00:01<00:00, 12.4MB/s]


In [7]:
!ls

digit-recognizer.zip  mnist_test.csv  mnist_train.csv  sample_data


In [8]:
# Распаковываем данные из архива
!unzip digit-recognizer.zip

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [9]:
import pandas as pd

# Чтение данных из MNIST test и train
mnist_test = pd.read_csv("/content/mnist_test.csv")
mnist_train = pd.read_csv("/content/mnist_train.csv")

sample_submission = pd.read_csv("/content/sample_submission.csv")

test = pd.read_csv("/content/test.csv")
train = pd.read_csv("/content/train.csv")

In [10]:
# Получение количества колонок (cols)
cols = test.columns

# Добавление данных в столбец dataset
test['dataset'] = 'test'
train['dataset'] = 'train'

In [11]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,dataset
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,train


In [12]:
# Этот код объединяет (метод concat) два датасета (train и test) и сохраняет их в новом датасете, который называется dataset.
# Параметр drop используется для удаления столбца ‘label’ из датасета train. Параметр axis=1 означает, что мы работаем со строками (по умолчанию столбцы).
# Метод reset_index используется для сброса индекса, чтобы можно было легко работать с данными.
dataset = pd.concat([train.drop('label', axis=1), test]).reset_index()

In [26]:
# Этот код объединяет датасеты mnist_train и mnist_test в один датасет mnist. Метод reset_index вызывается с параметром drop=True, чтобы сбросить индекс и удалить его.
# Затем берутся значения из столбца ‘label’ в датасете mnist и сохраняются в переменную labels.
# Далее столбец ‘label’ удаляется из датасета mnist методом drop. Параметр inplace указывает, что изменение должно произойти на месте, т.е. не создавать копию датасета.
# В конце меняются имена столбцов датасета mnist, для этого используется метод columns.

mnist = pd.concat([mnist_train, mnist_test]).reset_index(drop=True)
labels = mnist['label'].values
mnist.drop('label', axis=1, inplace=True)
mnist.columns = cols

print(mnist.columns)

Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)


In [14]:
# 1. `mnist` сортируется по значениям в каждом столбце и получаются индексы отсортированных строк. Они сохраняются в переменной `idx_mnist`.
# 2. `dataset` также сортируется по значениям в каждом столбце, соответствующим столбцам в `mnist`. Из отсортированного `dataset` выбираются значения столбца 'dataset' с помощью `['dataset'].values` и сохраняются в переменной `dataset_from`.
# 3. То же самое делается для столбца 'index' в `dataset`, и его значения сохраняются в переменной `original_idx`.

idx_mnist = mnist.sort_values(by=list(mnist.columns)).index
dataset_from = dataset.sort_values(by=list(mnist.columns))['dataset'].values
original_idx = dataset.sort_values(by=list(mnist.columns))['index'].values

In [20]:
dataset_from

array(['train', 'train', 'train', ..., 'train', 'train', 'train'],
      dtype=object)

In [22]:
# Выполняется цикл, в котором происходит проверка условия: если значение переменной `dataset_from[i]` равно строке `'test'`,
# то в датафрейме `sample_submission` значение столбца `'Label'` для строки с индексом `original_idx[i]` присваивается значение `labels[idx_mnist[i]]`.#
for i in range(len(idx_mnist)):
    if dataset_from[i] == 'test':
        sample_submission.loc[original_idx[i], 'Label'] = labels[idx_mnist[i]]

In [23]:
sample_submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [24]:
# Конвертация в csv
sample_submission.to_csv('submission2.csv', index=False)

# Отправляем решение на соревнование
!kaggle competitions submit -c digit-recognizer -m "submission for score 1" -f submission2.csv

In [25]:
# Отправляем решение на соревнование
!kaggle competitions submit -c digit-recognizer -m "submission for score 1" -f submission2.csv

100% 208k/208k [00:02<00:00, 82.4kB/s]
Successfully submitted to Digit Recognizer